# BalticMaterials GmbH
## AI-Camera Luxonis OAK-1 access
**Author:** Sven Nivera  
**Date:** 07.08.2023  
**Version:** 0.1.0  
**Description:** This notebook contains an example on how to access the OAK-1 camera via USB using opencv.

In [1]:
import cv2
import depthai as dai
import numpy as np

Every action regarding the camera system must be placed inside a pipeline. The camera is connected to an onboard chip with which the communication is made!

In [2]:
pipeline = dai.Pipeline()

The camera must be added to the stream and the resolution be set.

In [3]:
cam = pipeline.create(dai.node.ColorCamera) # add cam
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P) # set resolution

Communication between host and device is hadled by XLink. To access data from the device to host XLinkOut must be used.

In [4]:
xout_rgb = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
cam.preview.link(xout_rgb.input)

Frames are received as one-dimensional arrays. A custom conversion function is needed to produce viewable 2D Images.

In [5]:
def frameNorm(frame, bbox):
    normVals = np.full(len(bbox), frame.shape[0])
    normVals[::2] = frame.shape[1]
    return (np.clip(np.array(bbox), 0, 1) * normVals).astype(int)

When initialized a frame can be grapped from the device once it is ready. An converted output is shown via opencv.

In [ ]:
with dai.Device(pipeline) as device: # initialize device via USB3
    q_rgb = device.getOutputQueue("rgb") # frame queue for host side filling up with frames from the cam
    frame = None
    i = 0
    while True:
        in_rgb = q_rgb.tryGet() # Consuming latest (or None) frame from queue
        if in_rgb is not None:
            frame = in_rgb.getCvFrame() # Receive frame 
            if frame is not None:
                print(i)
                i += 1
                cv2.imshow("preview", frame)
                if cv2.waitKey(1) == ord('q'):
                    break